In [104]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [4]:
data_path = 'B.a.f/mirae/'
#mkt = pd.read_csv(data_path+'cs_mkkt_dataset/mkt_idx.csv',low_memory=False)
#cs = pd.read_csv(data_path+'cs_mkkt_dataset/cs_data.csv',low_memory=False)
#itm = pd.read_csv(data_path+'cs_mkkt_dataset/apy_itm_hist.csv',low_memory=False)

In [34]:
news=pd.read_csv("news.csv")

In [7]:
#news01.info()

In [8]:
#news01.head()

In [9]:
#news01["ITEM_NAME"]="마이크로컨텍솔"

In [121]:

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 경로 지정 (주피터 노트북에서 실행하는 경우에는 폰트 경로가 다를 수 있으니 적절하게 수정해주세요.)
font_path = 'NanumGothic.otf' 

# 폰트 설정
fontprop = fm.FontProperties(fname=font_path, size=12)

# 그래프에 폰트 설정 적용
plt.rcParams['font.family'] = fontprop.get_name()
plt.rcParams['axes.unicode_minus'] = False


In [11]:
#sns.countplot(y="SUBCATEGORY", data=news01)

In [12]:
#sns.kdeplot(y="IMPORTANCE", data=news01)

In [13]:
#news01["DATE_TIME"] = pd.to_datetime(news01["DATE_TIME"].str.replace("_","-"))

In [14]:
#태그 리스트, 아이템네임 변수랑 합치고, 제목 단어만 분리시킨 변수를 합쳐서 전체 관련 단어 변수 하나로 만들기

In [16]:
!pip install transformers

In [17]:
!pip install tensorflow_addons

In [20]:
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss
from transformers import BertTokenizerFast
import torch
from transformers import TFBertForSequenceClassification

In [21]:
MODEL_NAME = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
DATASET_URL = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
DATASET_NAME = "finance_data.csv"

In [21]:
urllib.request.urlretrieve(DATASET_URL,
                           filename = DATASET_NAME
                           )

('finance_data.csv', <http.client.HTTPMessage at 0x22305d41850>)

In [6]:
dataset = pd.read_csv(DATASET_NAME)
dataset.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [7]:
del dataset['sentence']

In [8]:
dataset['labels'] = dataset['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
dataset.head()

,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [9]:
# 입출력 데이터 분리
X_data = dataset['kor_sentence']
y_data = dataset['labels']

In [16]:
TEST_SIZE = 0.2 # Train: Test = 8 :2 분리
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                    test_size = TEST_SIZE,
                                                    random_state = RANDOM_STATE,
                                                    stratify = y_data)

In [17]:
MAX_SEQ_LEN = 64

In [22]:
def convert_data(X_data, y_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments, targets = [], [], [], []

    for X, y in tqdm(zip(X_data, y_data)):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)

        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros

        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0]*MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        targets.append(y)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

In [23]:
# train 데이터를 Bert의 Input 타입에 맞게 변환
train_x, train_y = convert_data(X_train, y_train)
# test 데이터를 Bert의 Input 타입에 맞게 변환
test_x, test_y = convert_data(X_test, y_test)

3876it [00:01, 2125.04it/s]
970it [00:00, 2589.74it/s]


In [24]:
# token, mask, segment 입력 정의
token_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_word_ids')
mask_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_masks')
segment_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_segment')
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

In [25]:
bert_output = bert_outputs[0]

In [26]:
DROPOUT_RATE = 0.5
NUM_CLASS = 3
dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(bert_output)
# Multi-class classification 문제이므로 activation function은 softmax로 설정
sentiment_layer = tf.keras.layers.Dense(NUM_CLASS, activation='softmax', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02))(dropout)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_layer)

In [27]:
OPTIMIZER_NAME = 'RAdam'
LEARNING_RATE = 5e-5
TOTAL_STEPS = 10000
MIN_LR = 1e-5
WARMUP_PROPORTION = 0.1
EPSILON = 1e-8
CLIPNORM = 1.0
optimizer = tfa.optimizers.RectifiedAdam(learning_rate = LEARNING_RATE,
                                          total_steps = TOTAL_STEPS,
                                          warmup_proportion = WARMUP_PROPORTION,
                                          min_lr = MIN_LR,
                                          epsilon = EPSILON,
                                          clipnorm = CLIPNORM)

In [28]:
sentiment_model.compile(optimizer = optimizer,
                        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                        metrics = ['accuracy'])

In [29]:
MIN_DELTA = 1e-3
PATIENCE = 5

early_stopping = EarlyStopping(
    monitor = "val_accuracy",
    min_delta = MIN_DELTA,
    patience = PATIENCE)

In [36]:
# 최고 성능의 모델 파일을 저장할 이름과 경로 설정
BEST_MODEL_NAME = './model/best_model.h5'
model_checkpoint = ModelCheckpoint(
    filepath = BEST_MODEL_NAME,
    monitor = "val_loss",
    mode = "min",
    save_best_only = True, # 성능 향상 시에만 모델 저장
    verbose = 1
)

In [37]:
callbacks = [early_stopping, model_checkpoint]

In [39]:
EPOCHS = 10
BATCH_SZIE = 32

sentiment_model.fit(train_x, train_y,
                    epochs = EPOCHS,
                    shuffle = True,
                    batch_size = BATCH_SZIE,
                    validation_data = (test_x, test_y),
                    callbacks = callbacks
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})                    )

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.0980 - accuracy: 0.2998 
Epoch 1: val_loss improved from inf to 1.09687, saving model to ./model\best_model.h5
122/122 [==============================] - 2428s 20s/step - loss: 1.0980 - accuracy: 0.2998 - val_loss: 1.0969 - val_accuracy: 0.2918
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 1.0938 - accuracy: 0.3344 
Epoch 2: val_loss improved from 1.09687 to 1.08551, saving model to ./model\best_model.h5
122/122 [==============================] - 2362s 19s/step - loss: 1.0938 - accuracy: 0.3344 - val_loss: 1.0855 - val_accuracy: 0.3454
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 1.0646 - accuracy: 0.4172 
Epoch 3: val_loss improved from 1.08551 to 1.02804, saving model to ./model\best_model.h5
122/122 [==============================] - 2662s 22s/step - loss: 1.0646 - accuracy: 0.4172 - val_loss: 1.0280 - val_accuracy: 0.3660
Epoch 4/10
122/122 [=====================

In [14]:
BEST_MODEL_NAME = './model/best_model.h5'

sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})                    

In [31]:
predicted_value = sentiment_model_best.predict(test_x)
predicted_label = np.argmax(predicted_value, axis = 1)

cl_report = classification_report(test_y, predicted_label, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(3)
cl_report_df.to_csv("CL_REPORT_FILE")
print(cl_report_df)

31/31 [==============================] - 202s 7s/step
              precision  recall  f1-score  support
0                 0.847   0.911     0.878  576.000
1                 0.775   0.733     0.753  273.000
2                 0.891   0.678     0.770  121.000
accuracy          0.832   0.832     0.832    0.832
macro avg         0.838   0.774     0.800  970.000
weighted avg      0.832   0.832     0.829  970.000


In [40]:
# 최고 성능의 모델 불러오기
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

In [32]:
inte = pd.read_csv('inte.csv')

In [42]:
len(inte)

332256

In [43]:
inte_x = inte["TITLE"]
inte_y = pd.Series(len(inte_x))

In [44]:
con_inte,others = convert_data(inte_x,inte_y)

1it [00:00,  5.44it/s]


In [45]:
predicted_value = sentiment_model_best.predict(con_inte)
predicted_label = np.argmax(predicted_value, axis = 1)

1/1 [==============================] - 10s 10s/step


In [46]:
predicted_value = sentiment_model_best.predict(test_x)
predicted_label = np.argmax(predicted_value, axis = 1)

31/31 [==============================] - 154s 5s/step


In [47]:
convert_data(inte)

TypeError: convert_data() missing 1 required positional argument: 'y_data'

In [ ]:
inte = pd.DataFrame(inte["TITLE"])
con_inte = convert_data(inte)
predicted_value = sentiment_model_best.predict(con_inte)
predicted_label = np.argmax(predicted_value, axis = 1)

inte["sentiment"] = predicted_value

In [ ]:
predicted_value = sentiment_model_best.predict(test_x)
predicted_label = np.argmax(predicted_value, axis = 1)

In [ ]:
CL_REPORT_FILE = "./metric/cl_report.csv"

cl_report = classification_report(test_y, predicted_label, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(3)
cl_report_df.to_csv(CL_REPORT_FILE)
print(cl_report_df)

In [ ]:
news01_x = news01["TITLE"]

In [35]:
news.head()

,DATE_TIME,TITLE,IMPORTANCE,CONTENT,NEWS_NUM,ITEM_NAME_ONLY,단축코드
0,20230101,"""이미 모든 악재 나왔다""…전문가가 뽑은 내년 뜰 업종은?",58.26,[머니투데이 홍순빈 기자] [[2023 증시 전망]국내 증시 주목할 업종]_x000...,1,고려아연,010130
1,20230101,"""이미 모든 악재 나왔다""…전문가가 뽑은 내년 뜰 업종은?",58.26,[머니투데이 홍순빈 기자] [[2023 증시 전망]국내 증시 주목할 업종]_x000...,1,영풍,000670
2,20230102,CES엔 대기업만 있나?…'삼성 C랩·LG NOVA' 톡톡 튀는 스타트업 향연,63.81,[머니투데이 한지연 기자] [[CES 2023 개막]⑦]_x000D_\n_x000D...,2,삼성전자,005930
3,20230102,CES엔 대기업만 있나?…'삼성 C랩·LG NOVA' 톡톡 튀는 스타트업 향연,63.81,[머니투데이 한지연 기자] [[CES 2023 개막]⑦]_x000D_\n_x000D...,2,LG전자,066570
4,20230102,"CBI ""지난해 최대 매출 기대…올해도 기록적인 성장세 전망""",58.19,[머니투데이 김건우 기자] _x000D_\n자동차 부품업체 CBI는 지난해 글로벌 ...,3,CBI,013720


In [56]:
news_con = news["CONTENT"]

In [62]:
news_con

0         [머니투데이 홍순빈 기자] [[2023 증시 전망]국내 증시 주목할 업종]_x000...
1         [머니투데이 홍순빈 기자] [[2023 증시 전망]국내 증시 주목할 업종]_x000...
2         [머니투데이 한지연 기자] [[CES 2023 개막]⑦]_x000D_\n_x000D...
3         [머니투데이 한지연 기자] [[CES 2023 개막]⑦]_x000D_\n_x000D...
4         [머니투데이 김건우 기자] _x000D_\n자동차 부품업체 CBI는 지난해 글로벌 ...
                                ...                        
518054    <p>KB증권에서 23일 삼성전자(005930)에 대해 "4분기 HBM3 공급 본격...
518055    <p>키움증권에서 23일 아모레퍼시픽(090430)에 대해 "기다림"라며 투자의견을...
518056    <p><strong>◆ Report briefing<br><br>안정성과 성장성을 ...
518057    <p style="text-align: justify;"><span style="f...
518058    HJ중공업(097230)은 공사수주에 관한 단일판매ㆍ공급계약체결을  23일에 공시했...
Name: CONTENT, Length: 518059, dtype: object

In [36]:
# 0과 1로 이루어진 랜덤 데이터 생성 (518059개)
random_data = np.random.randint(0, 2, size=518059)

# 시리즈 생성
series_data = pd.Series(random_data)

# 결과 확인
print(series_data)

0         1
1         1
2         1
3         0
4         1
         ..
518054    0
518055    0
518056    0
518057    0
518058    0
Length: 518059, dtype: int32


In [64]:
news_convert,fake = convert_data(news_con,series_data)
predicted_value = sentiment_model_best.predict(news_convert)
predicted_label = np.argmax(predicted_value, axis = 1)

news["sentiment"] = predicted_value

518059it [59:11, 145.86it/s]


16190/16190 [==============================] - 84472s 5s/step


ValueError: Expected a 1D array, got an array with shape (518059, 3)

In [73]:
predicted_label #중립:0 긍정:1 부정:2

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [74]:
news["sentiment"] = predicted_label

In [78]:
news.to_csv("predict_news")

In [77]:
##################################################################################여기까지가 BERT 모델링

,DATE_TIME,TITLE,IMPORTANCE,CONTENT,NEWS_NUM,ITEM_NAME_ONLY,단축코드,sentiment


In [ ]:
news_convert,fake = convert_data(news_con,series_data)
predicted_value = sentiment_model_best.predict(news_convert)
predicted_label = np.argmax(predicted_value, axis = 1)

news["sentiment"] = predicted_value

In [39]:
cs=pd.read_csv("inte.csv")
cs.head()

ParserError: Error tokenizing data. C error: out of memory